# Vanilla OTC Options

This notebook demonstrates how to access and use the functinalities of **ETI Options** which are part of our **QPS** module within LSEG Financial Analytics SDK.

## Imports

In [1]:
from lseg_analytics.instruments.options import (
    value,
    OptionDefinition,
    OptionDefinitionInstrument,
    OptionPricingParameters,
    UnderlyingDefinition,
    ExerciseDefinition,
    ScheduleDefinition
)

from lseg_analytics.common import AdjustableDate
from datetime import date
import pandas as pd
import json as js

## Pricing a Vanilla OTC Option

In [2]:
definition = OptionDefinition(
    underlying = UnderlyingDefinition(
        code="AAPL.O",
        underlying_type="Equity"
    ),
    exercise = ExerciseDefinition(
        strike=150,
        exercise_style="European",
        schedule = ScheduleDefinition(
            end_date = AdjustableDate(
                date = date(2025, 9, 13)
            )
        )
    ),
    option_type="Call"
)

pricing_parameters=OptionPricingParameters(
    valuation_date=date(2024, 12, 31)
)

response = value(
    definitions=[OptionDefinitionInstrument(definition=definition)],
    pricing_preferences=pricing_parameters
)

print(js.dumps(response.as_dict(), indent=4))

{
    "definitions": [
        {
            "definition": {
                "underlying": {
                    "underlyingType": "Equity",
                    "code": "AAPL.O"
                },
                "exercise": {
                    "strike": 150.0,
                    "schedule": {
                        "endDate": {
                            "dateType": "AdjustableDate",
                            "date": "2025-09-13"
                        }
                    },
                    "exerciseStyle": "European"
                },
                "optionType": "Call"
            }
        }
    ],
    "pricingPreferences": {
        "valuationDate": "2024-12-31"
    },
    "analytics": [
        {
            "description": {
                "instrumentCode": "",
                "underlyingRic": "AAPL.O",
                "instrumentDescription": "Cash_EURO_AAPL.O"
            },
            "valuation": {
                "volatility": {
                    "value":

In [3]:
valuation = response.analytics[0].valuation

def flatten(data):
    rows = []

    for key, section in data.items():
        row = {"metric": key}

        # Start with raw values
        value = section.get("value")
        unit = section.get("unit")
        percent = section.get("percent")

        # Handle dealCurrency override
        dc = section.get("dealCurrency")
        if dc is not None:
            value = getattr(dc, "value", value)
            unit = getattr(dc, "currency", unit)

        # Round and assign
        if value is not None:
            row["value"] = round(value, 3)
        if unit is not None:
            row["unit"] = unit
        if percent is not None:
            row["percent"] = round(percent, 3)

        rows.append(row)

    return pd.DataFrame(rows).set_index("metric")


flatten(valuation)

,value,unit,percent
metric,,,
volatility,36.456,Percentage,NaN
marketValue,102.257,USD,NaN
totalMarketValue,10225.700,USD,NaN
intrinsicValue,100.420,USD,NaN
timeValue,1.837,USD,NaN
premiumOverCash,1.837,USD,0.734
moneyness,166.947,Percentage,NaN
annualizedYield,0.010,Absolute,NaN
